In [1]:
import os
import sys


os.chdir("/teamspace/studios/this_studio/nurse-helper-chatbot")


sys.path.append(os.getcwd())

In [2]:
from llm import llm
from graph import graph
from langchain.chains import GraphCypherQAChain
from langchain.prompts.prompt import PromptTemplate


CYPHER_GENERATION_TEMPLATE = """Task: Generate a Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Ensure that the generated Cypher statements are syntactically correct and relevant to the question.
Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.

Do not return entire nodes or embedding properties.

Fine Tuning:

Example Cypher Statements:

1. To find the information for a specific hospital by Hospital Name
```
MATCH (h:hospital {{Hospital_Name: Hospital Name"}})
RETURN h.Hospital_Name, h.Hospital_Contact, h.Hospital_ID, h.Specialisation_Offered, h.State
```

2. To find the hospitals based on single, multiple, or alternative specializations:
```
MATCH (h:hospital)
WHERE h.Specialisation_Offered CONTAINS "Cardiology" OR h.Specialisation_Offered CONTAINS "Neurology"
RETURN 
    h.Hospital_ID AS HospitalID, 
    h.Hospital_Name AS HospitalName, 
    h.Specialisation_Offered AS Specialisations;

```

3.  To find the hospitals based on single, multiple, or alternative specializations:
```
MATCH (h:hospital)
WHERE h.Specialisation_Offered CONTAINS "Orthopedics" AND h.Specialisation_Offered CONTAINS "Pulmonology"
RETURN 
    h.Hospital_ID AS HospitalID, 
    h.Hospital_Name AS HospitalName, 
    h.Specialisation_Offered AS Specialisations;

```

4. To find the hospitals in a specific state:
```
   MATCH (h:hospital) 
   WHERE h.State CONTAINS "Penang" 
   RETURN h.Hospital_ID, h.Hospital_Name;
```

5. To find the physicians with a specific specialization recruited by a specific Hospital
```
MATCH (h:hospital {{Hospital_Name: "Penang General Hospital"}})-[:recruit]->(p:physician)
WHERE p.Specialization = "Pulmonology"
RETURN p.Physician_Name AS PhysicianName, 
       p.License_Number AS LicenseNumber, 
       p.Physician_Contact AS PhysicianContactNumber;
```

Schema:
{schema}

Question:
{question}

Cypher Query:
"""


hospital_cypher_prompt = PromptTemplate.from_template(CYPHER_GENERATION_TEMPLATE)

hospital_cypher_qa = GraphCypherQAChain.from_llm(
    llm,
    graph=graph,
    return_intermediate_steps=True,
    verbose=True,
    allow_dangerous_requests=True,
    validate_cypher=True,
    function_response_system="Response based on the Cypher Query and context",
    cypher_prompt=hospital_cypher_prompt
)

/teamspace/studios/this_studio/nurse-helper-chatbot/graph.py:7: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(


In [3]:
# Example query to invoke the chain
## can u give me the Penang General hospital info
## which hospital offer specialisation in Neurology?
## which hospitals locate at Penang?
## Penang General Hospital recruit physician with specialization in Pulmonology?
query = "which hospitals locate at Penang?"

# Invoke the chain with the complete input data
response = hospital_cypher_qa.invoke(query)

response



> Entering new GraphCypherQAChain chain...


Generated Cypher:

MATCH (h:hospital) 
WHERE h.State CONTAINS "Penang" 
RETURN h.Hospital_ID, h.Hospital_Name;

Full Context:
[{'h.Hospital_ID': 1, 'h.Hospital_Name': 'Penang General Hospital'}, {'h.Hospital_ID': 2, 'h.Hospital_Name': 'Sunway Medical Centre Penang'}, {'h.Hospital_ID': 3, 'h.Hospital_Name': 'Penang Adventist Hospital'}, {'h.Hospital_ID': 4, 'h.Hospital_Name': 'Island Hospital'}, {'h.Hospital_ID': 5, 'h.Hospital_Name': 'Gleneagles Hospital Penang'}, {'h.Hospital_ID': 6, 'h.Hospital_Name': 'Pantai Hospital Penang'}, {'h.Hospital_ID': 7, 'h.Hospital_Name': 'Loh Guan Lye Specialists Centre'}, {'h.Hospital_ID': 8, 'h.Hospital_Name': 'KPJ Penang Specialist Hospital'}, {'h.Hospital_ID': 9, 'h.Hospital_Name': 'Tropicana Medical Centre Penang'}, {'h.Hospital_ID': 10, 'h.Hospital_Name': 'Mount Miriam Cancer Hospital'}]

> Finished chain.


{'query': 'which hospitals locate at Penang?',
 'result': 'The hospitals located in Penang are Penang General Hospital, Sunway Medical Centre Penang, Penang Adventist Hospital, Island Hospital, Gleneagles Hospital Penang, Pantai Hospital Penang, Loh Guan Lye Specialists Centre, KPJ Penang Specialist Hospital, Tropicana Medical Centre Penang, and Mount Miriam Cancer Hospital.',
 'intermediate_steps': [{'query': '\nMATCH (h:hospital) \nWHERE h.State CONTAINS "Penang" \nRETURN h.Hospital_ID, h.Hospital_Name;\n'},
  {'context': [{'h.Hospital_ID': 1,
     'h.Hospital_Name': 'Penang General Hospital'},
    {'h.Hospital_ID': 2, 'h.Hospital_Name': 'Sunway Medical Centre Penang'},
    {'h.Hospital_ID': 3, 'h.Hospital_Name': 'Penang Adventist Hospital'},
    {'h.Hospital_ID': 4, 'h.Hospital_Name': 'Island Hospital'},
    {'h.Hospital_ID': 5, 'h.Hospital_Name': 'Gleneagles Hospital Penang'},
    {'h.Hospital_ID': 6, 'h.Hospital_Name': 'Pantai Hospital Penang'},
    {'h.Hospital_ID': 7, 'h.Hospita

In [8]:
from llm import llm
from graph import graph
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain.schema import StrOutputParser
from langchain.tools import Tool
from langchain_community.chat_message_histories import Neo4jChatMessageHistory
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain import hub
from utils import get_session_id
from tools.review import get_review
from langchain_community.callbacks.streamlit import StreamlitCallbackHandler
import streamlit as st 


tools = [

    Tool.from_function(
        name="Hospital Information",  
        description="Provide information about hospital questions using Cypher and context from GraphCypherQAChain",
        func=hospital_cypher_qa, 
    ),
    
]


def get_memory(session_id):
    return Neo4jChatMessageHistory(session_id=session_id, graph=graph)

agent_prompt = PromptTemplate.from_template("""
You are a nursing expert providing information about nursing care.
Be as helpful as possible and return as much information as possible.
Only answer questions related to checkups, hospitals, or patients.

You must rely solely on the information provided in the context and the tools available to you. 
Do not use any pre-trained knowledge.



TOOLS:
------

You have access to the following tools:

{tools}

To use a tool, please follow this format:
```
Thought: Do I need to use a tool? Yes 
Action: [the action to take, should be one of {tool_names}] 
Action Input: [the input to the action] 
Observation: [the result of the action]
```

You **must** use the tools provided for every relevant question. 
Do not decline to use a tool, as it is essential for generating accurate responses.

When generating responses:
Always follow the required format strictly.
Use the actual Cypher examples from the `GraphCypherQAChain` to construct accurate queries.
Validate the information retrieved from the tools before providing a final answer.
If the information is insufficient, indicate that further clarification is needed.

Use the following format to provide a response:
```
Final Answer: [your response here]
```

Begin!

Previous conversation history:
{chat_history}

New input: {input}
{agent_scratchpad}
""")

agent = create_react_agent(llm, tools, agent_prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    )

chat_agent = RunnableWithMessageHistory(
    agent_executor,
    get_memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)



def generate_response(user_input):
    """
    Create a handler that calls the Conversational agent
    and returns a response to be rendered in the UI
    """

    # Initialize the Streamlit callback handler
    st_callback = StreamlitCallbackHandler(st.container())

    # Call the agent and capture the response
    response = chat_agent.invoke(
        {"input": user_input},
        {"configurable": {"session_id": get_session_id()}, "callbacks": [st_callback]}
    )

    # Return the final output
    return response['output']

In [9]:
# Example query to invoke the chain
## can u give me the Penang General hospital info
## which hospital offer specialisation in Neurology?
## which hospitals locate at Penang?
## Penang General Hospital recruit physician with specialization in Pulmonology?
query = "Give the physician name which specialize in Pulmonology were recruited by Penang General Hospital?"

# Invoke the chain with the complete input data
response = generate_response(query)

response



2024-12-30 17:44:25.994 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:44:25.996 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:44:25.997 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:44:25.997 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:44:29.607 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:44:29.608 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.




> Entering new AgentExecutor chain...


2024-12-30 17:44:29.933 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:44:29.935 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:44:29.935 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:44:29.936 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:44:29.950 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:44:29.951 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:44:29.952 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:44:29.953 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

```
Thought: Do I need to use a tool? Yes 
Action: Hospital Information 
Action Input: {"hospital": "Penang General Hospital", "specialization": "Pulmonology"} 

> Entering new GraphCypherQAChain chain...


Error in StreamlitCallbackHandler.on_llm_end callback: StreamlitAPIException()


...

Error in StreamlitCallbackHandler.on_tool_end callback: StreamlitAPIException()


{'query': '{"hospital": "Penang General Hospital", "specialization": "Pulmonology"}', 'result': "I'm sorry, I don't know the answer.", 'intermediate_steps': [{'query': '\nMATCH (h:hospital {Hospital_Name: "Penang General Hospital"})-[:recruit]->(p:physician)\nWHERE p.Specialization = "Pulmonology"\nRETURN p.Physician_Name AS PhysicianName, \n       p.License_Number AS LicenseNumber, \n       p.Physician_Contact AS PhysicianContactNumber;\n'}, {'context': [{'PhysicianName': 'Dr. Azman bin Yusof', 'LicenseNumber': 12345, 'PhysicianContactNumber': '03-1234567'}, {'PhysicianName': 'Dr. Suraya binti Mohamad', 'LicenseNumber': 49619, 'PhysicianContactNumber': '03-6441914'}, {'PhysicianName': 'Dr. Low Mei Yen', 'LicenseNumber': 76246, 'PhysicianContactNumber': '03-7527747'}, {'PhysicianName': 'Dr. Loo Zhao Fen', 'LicenseNumber': 71172, 'PhysicianContactNumber': '03-3836337'}]}]}

2024-12-30 17:44:34.277 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:44:34.278 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:44:34.278 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:44:34.278 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:44:34.288 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:44:34.289 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:44:34.290 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:44:34.290 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

Final Answer: The physicians specializing in pulmonology recruited by Penang General Hospital are Dr. Azman bin Yusof (License Number: 12345, Contact: 03-1234567), Dr. Suraya binti Mohamad (License Number: 49619, Contact: 03-6441914), Dr. Low Mei Yen (License Number: 76246, Contact: 03-7527747), and Dr. Loo Zhao Fen (License Number: 71172, Contact: 03-3836337).

> Finished chain.


'The physicians specializing in pulmonology recruited by Penang General Hospital are Dr. Azman bin Yusof (License Number: 12345, Contact: 03-1234567), Dr. Suraya binti Mohamad (License Number: 49619, Contact: 03-6441914), Dr. Low Mei Yen (License Number: 76246, Contact: 03-7527747), and Dr. Loo Zhao Fen (License Number: 71172, Contact: 03-3836337).'